In [ ]:
import polars as pl
import duckdb
from datetime import datetime

### load data

In [3]:
con = duckdb.connect('./case-2.duckdb')

runners = con.sql("select * from pizza_runner.runners").pl()
pizza_names = con.sql("select * from pizza_runner.pizza_names").pl()
pizza_toppings = con.sql("select * from pizza_runner.pizza_toppings").pl()
runner_orders = con.sql("select * from pizza_runner.runner_orders").pl()
customer_orders = con.sql("select * from pizza_runner.customer_orders").pl()
pizza_recipes = con.sql("select * from pizza_runner.pizza_recipes").pl()

con.close()

### clean data

In [4]:
clean_customer_orders = customer_orders.with_columns(
    pl.when(pl.col('exclusions').is_in(['', 'null']))
      .then(pl.lit(None))
      .otherwise(pl.col('exclusions'))
      .alias('exclusions'),
    pl.when(pl.col('extras').is_in(['', 'null']))
      .then(pl.lit(None))
      .otherwise(pl.col('extras'))
      .alias('extras'),
)

#display(clean_customer_orders)

In [5]:
clean_runner_orders = runner_orders.with_columns(
    pl.when(pl.col('pickup_time').is_in(['', 'null']))
      .then(pl.lit(None))
      .otherwise(pl.col('pickup_time'))
      .str.to_datetime(format='%Y-%m-%d %H:%M:%S')
      .alias('pickup_time'),
    pl.when(pl.col('distance').is_in(['', 'null']))
      .then(pl.lit(None))
      .otherwise(pl.col('distance'))
      .str.replace(r"[^0-9\\.]*$", "")
      .cast(pl.Float64)
      .alias('distance'),
    pl.when(pl.col('duration').is_in(['', 'null']))
      .then(pl.lit(None))
      .otherwise(pl.col('duration'))
      .str.replace(r"[^0-9\\.]*$", "")
      .cast(pl.Int64)
      .alias('duration'),
    pl.when(pl.col('cancellation').is_in(['', 'null']))
      .then(pl.lit(None))
      .otherwise(pl.col('cancellation'))
      .alias('cancellation'),
)

#display(clean_runner_orders)

### analysis

#### A.1. pizzas ordered

In [10]:
(clean_customer_orders
    .select(pl.col('pizza_id').len().alias('pizzas_ordered'))
)

pizzas_ordered
u32
14


#### A.2. number of unique customer orders

In [9]:
(clean_customer_orders
    .select(pl.col('order_id').n_unique().alias('customer_orders'))
)

customer_orders
u32
10


#### A.3. number of successful order deliveries by each runner

In [66]:
(clean_runner_orders
    .filter(pl.col('cancellation').is_null())
    .group_by('runner_id')
    .agg(
        pl.col('duration').len().alias('orders_delivered')
    )
    .sort('runner_id')
)

runner_id,orders_delivered
i32,u32
1,4
2,3
3,1


#### A.4. deliveries by pizza type

In [16]:
(clean_runner_orders
    .join(clean_customer_orders, how='left', on='order_id')
    .filter(pl.col('cancellation').is_null())
    .group_by('pizza_id')
    .agg(pl.len().alias('deliveries'))
    .sort('pizza_id')
)

pizza_id,deliveries
i32,u32
1,9
2,3


#### A.5. pizza types ordered by each customer

In [11]:
(clean_customer_orders
    .group_by('customer_id', 'pizza_id')
    .agg(pl.len().alias('orders'))
    .join(pizza_names, how='left', on='pizza_id')
    .select('customer_id', 'pizza_name', 'orders')
    .sort('customer_id', 'pizza_name')
)

customer_id,pizza_name,orders
i32,str,u32
101,"""Meatlovers""",2
101,"""Vegetarian""",1
102,"""Meatlovers""",2
102,"""Vegetarian""",1
103,"""Meatlovers""",3
103,"""Vegetarian""",1
104,"""Meatlovers""",3
105,"""Vegetarian""",1


#### A.6. most pizzas delivered on a single order

In [14]:
(clean_runner_orders
    .filter(pl.col('cancellation').is_null())
    .join(clean_customer_orders, how='left', on='order_id')
    .group_by('order_id')
    .agg(pl.len().alias('pizzas_delivered'))
    .sort('pizzas_delivered', descending=True)
    .limit(1)
)

order_id,pizzas_delivered
i32,u32
4,3


#### A.7. changed vs unchanged pizzas delivered per customer

In [17]:
(clean_runner_orders
    .filter(pl.col('cancellation').is_null())
    .join(clean_customer_orders, how='left', on='order_id')
    .group_by('customer_id')
    .agg(
        (pl.col('exclusions').is_null() & pl.col('extras').is_null()).sum().alias('unchanged'),
        (pl.col('exclusions').is_not_null() | pl.col('extras').is_not_null()).sum().alias('changed'),
    )
    .sort('customer_id')
)

customer_id,unchanged,changed
i32,u32,u32
101,2,0
102,3,0
103,0,3
104,1,2
105,0,1


#### A.8. delivered with both exclusions and extras

In [35]:
(clean_runner_orders
    .join(clean_customer_orders, how='left', on='order_id')
    .filter(pl.col('cancellation').is_null() & pl.col('exclusions').is_not_null() & pl.col('extras').is_not_null())
    .select(pl.len().alias('pizzas_w_exclusions_and_extras'))
)

pizzas_w_exclusions_and_extras
u32
1


#### A.9. pizzas ordered by hour of the day

In [61]:
(pl.DataFrame({'hour_of_day': list(range(24))})
   .join(
       (clean_customer_orders
           .select(
               pl.col('order_id'),
               pl.col('order_time').dt.hour().alias('hour_of_day'),
           )
           .group_by('hour_of_day')
           .agg(pl.len().alias('orders'))
       ),
       how='left',
       on='hour_of_day',
   )
   .select(
       pl.col('hour_of_day'),
       pl.col('orders').fill_null(0).alias('pizza_orders'),
   )
   .filter(pl.col('hour_of_day') > 7)
)

hour_of_day,pizza_orders
i64,u32
8,0
9,0
10,0
11,1
12,0
…,…
19,1
20,0
21,3


#### A.10. orders by day of the week

In [62]:
(pl.DataFrame({'day_of_week': range(7)})
   .join(
       (clean_customer_orders
           .select(
               pl.col('order_id'),
               pl.col('order_time').dt.weekday().alias('day_of_week')
           )
           .unique()
           .group_by('day_of_week')
           .agg(pl.len().alias('orders'))
       ),
       how='left',
       on='day_of_week',
   )
   .select(
       pl.col('day_of_week'),
       pl.col('orders').fill_null(0),
   )
   .join(
       pl.DataFrame({
           'day_of_week': list(range(7)),
           'name': ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'],
       }),
       how='left',
       on='day_of_week',
   )
   .sort('day_of_week')
   .select('name', 'orders')
)

name,orders
str,u32
"""Sun""",0
"""Mon""",0
"""Tue""",0
"""Wed""",5
"""Thu""",2
"""Fri""",1
"""Sat""",2


#### B.1. runners signed up for each one-week period

In [ ]:
(runners
    .select(
        pl.col('runner_id'),
        ((pl.col('registration_date') - pl.lit(datetime(2021, 1, 1))).dt.total_days() // 7).alias('week'),
    )
    .group_by('week')
    .agg(pl.len().alias('runners_registered'))
    .sort('week')
)

week,runners_registered
i64,u32
0,2
1,1
2,1


#### B.2. average pickup time (minutes) per runner

In [42]:
(clean_runner_orders
    .join(clean_customer_orders, how='left', on='order_id')
    .filter(pl.col('cancellation').is_null())
    .select(
        pl.col('runner_id'),
        ((pl.col('pickup_time') - pl.col('order_time')).dt.total_seconds() / 60.0).alias('pickup_time_mins'),
    )
    .group_by('runner_id')
    .agg(pl.col('pickup_time_mins').mean().round(2))
    .sort('runner_id')
)

runner_id,pickup_time_mins
i32,f64
1,15.68
2,23.72
3,10.47


#### B.3. relationship between number of pizzas and order prep time

In [60]:
(clean_customer_orders
    .join(clean_runner_orders, how='left', on='order_id')
    .filter(pl.col('cancellation').is_null())
    .select(
        pl.col('order_id'),
        ((pl.col('pickup_time') - pl.col('order_time')).dt.total_seconds() / 60.0).alias('pickup_time_minutes')
    )
    .group_by('order_id')
    .agg(
        pl.len().alias('pizzas'),
        pl.col('pickup_time_minutes').mean().round(2),
    )
    .drop('order_id')
    .group_by('pizzas')
    .agg(pl.col('pickup_time_minutes').mean().round(1))
    .sort('pizzas')
)

pizzas,pickup_time_minutes
u32,f64
1,12.4
2,18.4
3,29.3
